In [158]:
import importlib
import corona_plots
importlib.reload(corona_plots)

#corona_plots.CoronaPlots.plot_global_cases(auto_open=True)
corona_plots.CoronaPlots.plot_usa_cases(auto_open=True)
#corona_plots.CoronaPlots.plot_usa_pos_neg_tracking(auto_open=True)
pass

xxx


""
date
1/22/20
1/23/20
1/24/20
1/25/20
1/26/20
...
3/20/20
3/21/20
3/22/20


ddd
